# Web Scraping

#### La información obtenida de Open Food Facts será completada con la información real de Abril de 2020 sobre precios de estos mismos alimentos.


In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
import re
import numpy as np
import datetime
import pymysql
from sqlalchemy import create_engine

Me traigo la lista de códigos de barras del df original (por diversos problemas)

In [5]:
df = pd.read_csv('/Users/Natalio/Desktop/Cibeles/Ironhack/Proyecto_Final/Recursos/en.openfoodfacts.org.products.csv', sep='\t')
codes = df['code'].to_list()
barcodes = [i for i in codes if len(str(i)) == 13]


In [7]:
len(barcodes)

934022

In [9]:
engine = create_engine('sqlite:///foodprices_dia.db')

In [ ]:
#def scrapinginfo (barcode):
 #   url = 'https://www.dia.es/compra-online/search?text=' + barcode + '&x=0&y=0'
  #  html = requests.get(url).content
   # bs = BeautifulSoup(html, 'lxml')
    #searching = bs.find_all('p', {'class':'price'})
    #price = [re.findall('.*(\d+\,\d+)\s\€', str(searching))[0]]
    #result = zip(price, barcode)
    #results = list(result)
    #print(result)

#scrapinginfo(str(5449000000996))

In [ ]:
#table = bs.find_all('p', {'class':'price'})
#type(table)

In [ ]:
#table = map(str, table)
#type(table)

In [ ]:
#javi = [re.findall('.*(\d+\,\d+)\s\€', e.text) for e in table]
#type(javi)

In [ ]:
#javi

In [ ]:
# zip map
#url = '5449000000996'
#price = hola + ';' + url
#price

In [ ]:
#df = pd.DataFrame(price, columns = ['price', 'code'])
#df

Nuevas pruebas para verificar el correcto funcionamiento

In [ ]:
barcodes = ['5449000000996','0','0000000000208']
dfprice = []
dfpricekg = []
for barcode in barcodes:
    url = 'https://www.dia.es/compra-online/search?text=' + barcode + '&x=0&y=0'
    html = requests.get(url).content
    bs = BeautifulSoup(html, 'lxml')

    try:
        searchingprice = bs.find_all('p', {'class':'price'})
        price = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingprice))[0])
        dfprice.append(price)

        searchingpricerperkg = bs.find_all('p', {'class':'pricePerKilogram'})
        pricekg = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingpricerperkg))[0])
        dfpricekg.append(pricekg)
    
    except:
        dfprice.append('no data')
        dfpricekg.append('no data')
        

print(dfprice)
print(dfpricekg)
result = list(zip(barcodes, dfprice, dfpricekg))


df = pd.DataFrame(result, columns = ['barcode','price','pricekg'])

df

### Código final

In [ ]:
df = pd.read_csv('/Users/Natalio/Desktop/Cibeles/Ironhack/Proyecto_Final/Recursos/en.openfoodfacts.org.products.csv', sep='\t')
codes = df['code'].to_list()
barcodes = [i for i in codes if len(str(i)) == 13]

For problems with the result, I will divide the original list into n parts 

In [ ]:
# bc1 = np.array_split(barcodes,10)[0]
bc1

In [47]:
# starttime = datetime.datetime.now()
print(starttime)
barcodes = ['5449000000996','0','0000000000208']
dfprice = []
dfpricekg = []
count = 0
for barcode in barcodes:
    count += 1
    print(f'Scraping barcode: {barcode} ------------------------------------------------ Scrapeo número: {count} ')
    #if barcode in sql:
    try:
        url = 'https://www.dia.es/compra-online/search?text=' + str(barcode) + '&x=0&y=0'
        html = requests.get(url).content
        bs = BeautifulSoup(html, 'lxml')
        print('Recovering HTML')

        try:
            print('Parsing price and price per kg')
            searchingprice = bs.find_all('p', {'class':'price'})
            price = [str(re.findall('.*(\d+\,\d+)\s\€', str(searchingprice))[0])]
            print(price)
            
            
            searchingpricerperkg = bs.find_all('p', {'class':'pricePerKilogram'})
            pricekg = [str(re.findall('.*(\d+\,\d+)\s\€', str(searchingpricerperkg))[0])]
            print(pricekg)
            
            #result = list(zip(barcode, price, pricekg))
            #print(f'result {result}')
            #df = pd.DataFrame(result, columns = ['code','price_dia','pricekg_dia'])
            #df.to_sql('foodprices_dia', engine, if_exists = 'append', index=False)
            
    
    
        except:
            print('Could not find the codebar')
            price = ['null']
            pricekg = ['null']
        
        finally:
    
            result = list(zip(barcodes, price, pricekg))
            print(f'result {result}')
            df = pd.DataFrame(result, columns = ['code','price_dia','pricekg_dia'])
            df.to_sql('foodprices_dia', engine, if_exists = 'append', index=False)
   
        
        
    except:
        print('Coult not find the HTML request')
        pass
        
    
endtime = datetime.datetime.now()
duration = endtime - starttime
print(f'El proceso finaliza {endtime}.\nLa duración total es: {duration}')


2020-04-18 11:52:04.927447
Scraping barcode: 5449000000996 ------------------------------------------------ Scrapeo número: 1 
Recovering HTML
Parsing price and price per kg
['0,65']
['1,97']
result [('5449000000996', '0,65', '1,97')]
Scraping barcode: 0 ------------------------------------------------ Scrapeo número: 2 
Recovering HTML
Parsing price and price per kg
['1,09']
['5,52']
result [('5449000000996', '1,09', '5,52')]
Scraping barcode: 0000000000208 ------------------------------------------------ Scrapeo número: 3 
Recovering HTML
Parsing price and price per kg
Could not find the codebar
result [('5449000000996', 'null', 'null')]
El proceso finaliza 2020-04-18 12:39:13.776675.
La duración total es: 0:47:08.849228


In [48]:
df2 = pd.read_sql_query('SELECT * FROM foodprices_dia', engine)
df2

,code,price_dia,pricekg_dia
0,5449000000996,"0,65","1,97"
1,5449000000996,"1,09","5,52"
2,5449000000996,null,null
3,5,0,1
4,4,",",","
5,4,6,9
6,9,5,7
7,0,1,5
8,0,null,null
9,5,"0,65","1,97"


In [ ]:
def scrapinginfo(barcodes):
    dfprice = []
    dfpricekg = []
    for barcode in barcodes:
        url = 'https://www.dia.es/compra-online/search?text=' + barcode + '&x=0&y=0'
        html = requests.get(url).content
        bs = BeautifulSoup(html, 'lxml')

        try:
            searchingprice = bs.find_all('p', {'class':'price'})
            price = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingprice))[0])
            dfprice.append(price)

            searchingpricerperkg = bs.find_all('p', {'class':'pricePerKilogram'})
            pricekg = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingpricerperkg))[0])
            dfpricekg.append(pricekg)
    
        except:
            pass
        
    result = list(zip(barcodes, dfprice, dfpricekg))

    df = pd.DataFrame(result, columns = ['code','price','pricekg'])
    df.to_csv('/Users/Natalio/Desktop/Cibeles/Ironhack/Proyecto_Final/Recursos/bc1.csv', sep='\t')

    return df

scrapinginfo(bc1)